## Content-Based Recommendation System

Content-Based systems recommends items to the customer similar to previously high-rated items by the customer. It uses the features and properties of the item. From these properties, it can calculate the similarity between the items.

In a content-based recommendation system, first, we need to create a profile for each item, which represents the properties of those items. From the user profiles are inferred for a particular user. We use these user profiles to recommend the items to the users from the catalog.

In [0]:
%scala
val filepath1= "abfss://.../mldata/MoviesDataRecommendation/movies_metadata.csv"
var df1=spark.read.format("csv").option("header", "true").option("delimiter", ",").load(filepath1)
df1.createOrReplaceTempView("movies_metadata")

filepath1: String = abfss://adlsstore@psinsightsadlsdev.dfs.core.windows.net/mldata/MoviesDataRecommendation/movies_metadata.csv
df1: org.apache.spark.sql.DataFrame = [adult: string, belongs_to_collection: string ... 22 more fields]

In [0]:
#Convert spark sql df to pyspark df

#For memory saving mode: consider only 10 movies 
df_pyspark_movies= spark.sql("""select original_title,overview from movies_metadata""")  

df_movies  = df_pyspark_movies.toPandas()

#persist for handy use
outdir = '/dbfs/FileStore/df_movies_metadata.csv'
df_movies.to_csv(outdir, index=False)
#input_dataframe = pd.read_csv("/dbfs/FileStore/Dataframe.csv", header='infer')

In [0]:
import pandas as pd
df_movies_metadata = pd.read_csv("/dbfs/FileStore/df_movies_metadata.csv", header='infer')
df_movies_metadata

Out[10]:

,original_title,overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"""Cheated on, mistreated and stepped on, the wo..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...
...,...,...
45567,رگ خواب,Rising and falling between a man and woman.
45568,Siglo ng Pagluluwal,An artist struggles to finish his work while a...
45569,Betrayal,"When one of her hits goes wrong, a professiona..."
45570,Satana likuyushchiy,"In a small town live two brothers, one a minis..."


In [0]:
#Now we will transform the overview column in the vector form so that we can compute similarity. Use the below code to convert it.  We have used TFidfVectorizer for the same. 
#TfidfVectorizer – Transforms text to feature vectors that can be used as input to estimator. vocabulary_ Is a dictionary that converts each token (word) to feature index in the matrix, each unique token gets a feature index.
#Check if any nulls then drop it
print("Drop NULLS as it cannot fit for TF-IDF transform:\n",df_movies_metadata.isnull().sum())
df_movies_metadata.dropna(inplace=True)
print("After NULLS dropped, checking NULL count:\n",df_movies_metadata.isnull().sum())
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english', lowercase=True)
matrix = tf.fit_transform(df_movies_metadata['overview'])
#matrix


#Now we are ready to compute cosine similarity to check what all movies are of the same content on the basis of the overview column that was present in the data set. 
#Get the similarity of each Movie(i in 0-n) with all other Movies(0-n)
from sklearn.metrics.pairwise import linear_kernel
cosine_similarities = linear_kernel(matrix,matrix)
#cosine_similarities


#Indexing all Movie titles
movie_title = df_movies_metadata['original_title']
indices = pd.Series(df_movies_metadata.index, index=df_movies_metadata['original_title'])


#Get the 'k_top' similar movies with passed movie 'original_title'
def movie_recommend(original_title,k_top):
  idx = indices[original_title]
  sim_scores = list(enumerate(cosine_similarities[idx]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[1:k_top+1]
  movie_indices = [i[0] for i in sim_scores]
  return movie_title.iloc[movie_indices]

movie_recommend('Toy Story',10)


Drop NULLS as it cannot fit for TF-IDF transform:
 original_title 0
overview 0
dtype: int64
After NULLS dropped, checking NULL count:
 original_title 0
overview 0
dtype: int64
Out[12]: 3000 Toy Story 2
20719 18 Fingers of Death!
15370 Toy Story 3
24573 Small Fry
10313 The 40 Year Old Virgin
23881 Andy Hardy's Blonde Trouble
38568 Superstar: The Life and Times of Andy Warhol
29268 Hot Splash
42819 Andy Peters: Exclamation Mark Question Point
39705 Ozzie
Name: original_title, dtype: object